# 🔐 Face & Gesture-Based Authentication System
This notebook implements a multi-factor authentication system using face recognition, hand gestures, and OTP-based recovery.

---

In [1]:
#import necessary libraries, modules and packages
import cv2
import numpy as np
import os
import pandas as pd
import time
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import warnings
import webbrowser
import subprocess
import sys
from http.server import HTTPServer, BaseHTTPRequestHandler
import threading
import smtplib
import random
import string
from email.mime.text import MIMEText
import getpass
import logging
import mediapipe as mp
import re

In [2]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    filename='auth_system.log',
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [3]:
# Initialize MediaPipe solutions
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [4]:
# Create directories if they don't exist
os.makedirs('user_data', exist_ok=True)
os.makedirs('user_data/face_encodings', exist_ok=True)
os.makedirs('models', exist_ok=True)

In [5]:
# Initialize Excel file
def init_files():
    if not os.path.exists('user_data/users.xlsx'):
        df = pd.DataFrame(columns=['username', 'email', 'gesture_sequence', 'face_encoding_file', 'is_admin'])
        df.to_excel('user_data/users.xlsx', index=False)

init_files()

In [6]:
# Gesture recognition using MediaPipe Hands
GESTURE_NAMES = {
    0: "fist",
    1: "thumbs_up",
    2: "peace",
    4: "point_up",
    5: "open_hand",
    6: "call_me",
    7: "rock_on",
    8: "victory",
}

GESTURE_INSTRUCTIONS = {
    "fist": "Make a fist with your whole hand",
    "thumbs_up": "Extend your thumb upwards with other fingers closed",
    "peace": "Extend your index and middle fingers (V sign)",
    "point_up": "Extend your index finger upwards with other fingers closed",
    "open_hand": "Extend all fingers with palm facing camera",
    "call_me": "Extend thumb and pinky with other fingers closed (like a phone)",
    "rock_on": "Extend index and pinky fingers with middle/ring fingers closed",
    "victory": "Extend index and middle fingers spread apart (peace sign with space)",
}

In [7]:
#building landmark model for gesture recognition
def extract_hand_landmarks(frame, hands):
    """Extract hand landmarks using MediaPipe"""
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    
    if not results.multi_hand_landmarks:
        return None
    
    landmarks = results.multi_hand_landmarks[0].landmark
    return [[landmark.x, landmark.y, landmark.z] for landmark in landmarks]

In [8]:
# Gesture recognition function that uses the extracted landmarks to identify gestures
def recognize_gesture(landmarks):
    """Recognize gestures based on hand landmarks"""
    if not landmarks or len(landmarks) < 21:
        return None
    
    thumb_tip = landmarks[4]
    index_tip = landmarks[8]
    middle_tip = landmarks[12]
    ring_tip = landmarks[16]
    pinky_tip = landmarks[20]
    wrist = landmarks[0]
    
    def distance(a, b):
        return np.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)
    
    thumb_dist = distance(thumb_tip, wrist)
    index_dist = distance(index_tip, wrist)
    middle_dist = distance(middle_tip, wrist)
    ring_dist = distance(ring_tip, wrist)
    pinky_dist = distance(pinky_tip, wrist)
    
    def angle(a, b, c):
        ba = np.array(a) - np.array(b)
        bc = np.array(c) - np.array(b)
        cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        return np.arccos(cosine)
    
    if (index_dist < 0.15 and middle_dist < 0.15 and 
        ring_dist < 0.15 and pinky_dist < 0.15 and
        thumb_dist < 0.15):
        return "fist"
    
    if (thumb_dist > 0.2 and 
        index_dist < 0.15 and middle_dist < 0.15 and 
        ring_dist < 0.15 and pinky_dist < 0.15):
        return "thumbs_up"
    
    if (index_dist > 0.2 and middle_dist > 0.2 and 
        ring_dist < 0.15 and pinky_dist < 0.15 and
        angle(index_tip, middle_tip, wrist) < 1.0):
        return "peace"
    
    if (index_dist > 0.25 and 
        middle_dist < 0.15 and ring_dist < 0.15 and pinky_dist < 0.15):
        return "point_up"
    
    if (index_dist > 0.2 and middle_dist > 0.2 and 
        ring_dist > 0.2 and pinky_dist > 0.2 and
        thumb_dist > 0.2):
        return "open_hand"
    
    if (thumb_dist > 0.2 and pinky_dist > 0.2 and 
        index_dist < 0.15 and middle_dist < 0.15 and ring_dist < 0.15):
        return "call_me"
    
    if (index_dist > 0.2 and pinky_dist > 0.2 and 
        middle_dist < 0.15 and ring_dist < 0.15):
        return "rock_on"
    
    if (index_dist > 0.2 and middle_dist > 0.2 and 
        ring_dist < 0.15 and pinky_dist < 0.15 and
        angle(index_tip, middle_tip, wrist) > 1.5):
        return "victory"
    
    return None

In [9]:
# Function to extract face features using MediaPipe Face Mesh
def extract_face_features(frame, face_mesh):
    """Extract face features using MediaPipe Face Mesh"""
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)
    
    if not results.multi_face_landmarks:
        return None
    
    landmarks = results.multi_face_landmarks[0].landmark
    features = []
    for landmark in landmarks:
        features.extend([landmark.x, landmark.y, landmark.z])
    
    return features

In [10]:
# Function to send OTP to user's email
def send_otp_email(to_email, otp):
    """Send OTP to user's email"""
    sender_email = "email"
    sender_password = "password"   # Use App Password if 2FA is on
    subject = "Your OTP for Gesture Sequence Access"
    body = f"Your OTP is: {otp}\nThis OTP is valid for 5 minutes."

    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = sender_email
    msg['To'] = to_email

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, to_email, msg.as_string())
        print("✅ OTP sent successfully to your email!")
        return True
    except Exception as e:
        print(f"❌ Error sending OTP email: {e}")
        return False

In [11]:
def generate_otp():
    otp = random.randint(100000, 999999)  # Generate a random 6-digit OTP
    otp = str(otp)
    return otp

In [12]:
# Function to update the user database when a new user is added or an existing user is modified
def update_face_recognition_model():
    """Train or update the face recognition model"""
    try:
        users_df = pd.read_excel('user_data/users.xlsx')
    except Exception as e:
        print(f"Error reading user database: {e}")
        logging.error(f"Error reading user database: {e}")
        return
    
    if len(users_df) < 2:
        print("Need at least 2 users to train the face recognition model.")
        logging.warning("Insufficient users for face recognition model training")
        return
    
    known_face_features = []
    known_face_names = []
    
    for _, row in users_df.iterrows():
        try:
            with open(row['face_encoding_file'], 'rb') as f:
                features_list = pickle.load(f)
                known_face_features.extend(features_list)
                known_face_names.extend([row['username']] * len(features_list))
        except Exception as e:
            print(f"Error loading features for {row['username']}: {e}")
            logging.error(f"Error loading features for {row['username']}: {e}")
    
    if not known_face_features or len(set(known_face_names)) < 2:
        print("Not enough data to train face recognition model.")
        logging.warning("Insufficient data for face recognition model training")
        return
    
    le = LabelEncoder()
    labels = le.fit_transform(known_face_names)
    
    clf = SVC(kernel='linear', probability=True)
    clf.fit(known_face_features, labels)
    
    model_path = "models/face_recognition_model.pkl"
    try:
        with open(model_path, 'wb') as f:
            pickle.dump((le, clf), f)
        print("Face recognition model updated successfully!")
        logging.info("Face recognition model updated")
    except Exception as e:
        print(f"Error saving face recognition model: {e}")
        logging.error(f"Error saving face recognition model: {e}")

In [13]:
# Function to register a new user with face, email, gesture sequence, and admin status
def register_user():
    """Register new user with face, email, gesture sequence, and admin status"""
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open camera.")
        logging.error("Camera failed to open during user registration")
        return False
    
    try:
        users_df = pd.read_excel('user_data/users.xlsx')
    except Exception as e:
        print(f"Error reading user database: {e}")
        logging.error(f"Error reading user database: {e}")
        return False
    
    username = input("Enter new username: ").strip()
    if username in users_df['username'].values:
        print("Username already exists!")
        logging.warning(f"Registration attempt with existing username: {username}")
        return False
    
    email = input("Enter your email address: ").strip()
    if not email or '@' not in email or '.' not in email:
        print("Invalid email address!")
        logging.warning(f"Invalid email address provided: {email}")
        return False
    if email in users_df['email'].values:
        print("Email already registered!")
        logging.warning(f"Email already registered: {email}")
        return False
    email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    if not re.match(email_pattern, email):
        print("Invalid email format!")
        return False
    
    is_admin = input("Register as admin? (y/n): ").strip().lower() == 'y'
    
    print("\nFace Registration - Show your face clearly (front-facing)")
    face_samples = []
    sample_count = 0
    
    with mp_face_mesh.FaceMesh(
        static_image_mode=False,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5) as face_mesh:
        
        while sample_count < 10:
            ret, frame = cap.read()
            if not ret:
                continue
            
            frame = cv2.flip(frame, 1)
            display_frame = frame.copy()
            
            results = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    mp_drawing.draw_landmarks(
                        image=display_frame,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
                
                cv2.putText(display_frame, f"Face sample {sample_count+1}/10 - Press 'c'", 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                
                key = cv2.waitKey(1)
                if key == ord('c'):
                    features = extract_face_features(frame, face_mesh)
                    if features:
                        face_samples.append(features)
                        sample_count += 1
                        print(f"Captured face sample {sample_count}/10")
            else:
                cv2.putText(display_frame, "Show face to camera", 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            cv2.imshow("Face Registration", display_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    if sample_count < 10:
        print("Face registration incomplete!")
        logging.error(f"Face registration incomplete for {username}")
        cap.release()
        cv2.destroyAllWindows()
        return False
    
    encoding_file = f"user_data/face_encodings/{username}.pkl"
    try:
        with open(encoding_file, 'wb') as f:
            pickle.dump(face_samples, f)
    except Exception as e:
        print(f"Error saving face encodings: {e}")
        logging.error(f"Error saving face encodings for {username}: {e}")
        return False
    
    print("\nCreate your 5-gesture sequence from these options:")
    print("\n".join([f"{k}: {v} - {GESTURE_INSTRUCTIONS[v]}" for k, v in GESTURE_NAMES.items()]))
    print("\nYou will need to perform each gesture for 2 seconds to register it.")
    
    gesture_sequence = []
    
    with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=1,
        min_detection_confidence=0.7,
        min_tracking_confidence=0.5) as hands:
        
        while len(gesture_sequence) < 5:
            ret, frame = cap.read()
            if not ret:
                continue
            
            frame = cv2.flip(frame, 1)
            display_frame = frame.copy()
            
            landmarks = extract_hand_landmarks(frame, hands)
            
            if landmarks:
                hand_results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                if hand_results.multi_hand_landmarks:
                    hand_landmarks = hand_results.multi_hand_landmarks[0]
                    mp_drawing.draw_landmarks(
                        display_frame, 
                        hand_landmarks, 
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style())
                
                current_gesture = recognize_gesture(landmarks)
                
                if current_gesture:
                    cv2.putText(display_frame, f"Gesture Recognized: {current_gesture}", 
                               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    
                    key = cv2.waitKey(1)
                    if key == ord('c'):
                        gesture_sequence.append(current_gesture)
                        print(f"Added gesture {len(gesture_sequence)}: {current_gesture}")
                        time.sleep(0.5)
            
            cv2.putText(display_frame, f"Gesture {len(gesture_sequence)+1}/5 - Press 'c'", 
                       (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            cv2.imshow("Gesture Registration", display_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    cap.release()
    cv2.destroyAllWindows()
    
    if len(gesture_sequence) < 5:
        print("Gesture registration incomplete!")
        logging.error(f"Gesture registration incomplete for {username}")
        return False
    
    new_user = pd.DataFrame([{
        'username': username,
        'email': email,
        'gesture_sequence': str(gesture_sequence),
        'face_encoding_file': encoding_file,
        'is_admin': is_admin
    }])
    
    try:
        users_df = pd.concat([users_df, new_user], ignore_index=True)
        users_df.to_excel('user_data/users.xlsx', index=False)
    except Exception as e:
        print(f"Error saving user data: {e}")
        logging.error(f"Error saving user data for {username}: {e}")
        return False
    
    try:
        users_df = pd.read_excel('user_data/users.xlsx')
        if len(users_df) >= 2:
            update_face_recognition_model()
    except Exception as e:
        print(f"Error updating face recognition model: {e}")
        logging.error(f"Error updating face recognition model: {e}")
    
    role = "Admin" if is_admin else "User"
    print(f"\nRegistration successful! Registered as {role}.")
    print("Your gesture sequence:")
    print(" -> ".join(gesture_sequence))
    print("\nRemember this sequence as you'll need to perform it exactly during authentication.")
    logging.info(f"User {username} registered as {role}")
    return True

In [14]:
def load_face_recognition_model():
    """Load the trained face recognition model"""
    model_path = "models/face_recognition_model.pkl"
    if not os.path.exists(model_path):
        logging.warning("Face recognition model not found")
        return None, None
    
    try:
        with open(model_path, 'rb') as f:
            le, clf = pickle.load(f)
        return le, clf
    except Exception as e:
        print(f"Error loading face recognition model: {e}")
        logging.error(f"Error loading face recognition model: {e}")
        return None, None

In [15]:
# Function to authenticate admin using face recognition and gesture sequence
def authenticate_admin():
    """Authenticate admin with face recognition and gesture sequence"""
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open camera.")
        logging.error("Camera failed to open during admin authentication")
        return None
    
    try:
        users_df = pd.read_excel('user_data/users.xlsx')
    except Exception as e:
        print(f"Error reading user database: {e}")
        logging.error(f"Error reading user database: {e}")
        return None
    
    if users_df.empty:
        print("No users registered yet!")
        logging.warning("No users registered for admin authentication")
        return None
    
    admin_df = users_df[users_df['is_admin'] == True]
    if admin_df.empty:
        print("No admins registered!")
        logging.warning("No admins registered")
        return None
    
    le, clf = load_face_recognition_model()
    
    if le is None or clf is None:
        print("Face recognition model not trained yet!")
        logging.warning("Face recognition model not trained")
        return None
    
    username = None
    
    print("Admin Face Authentication - Show your face clearly")
    face_detected = False
    start_time = time.time()
    
    with mp_face_mesh.FaceMesh(
        static_image_mode=False,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5) as face_mesh:
        
        while time.time() - start_time < 60:
            ret, frame = cap.read()
            if not ret:
                continue
            
            frame = cv2.flip(frame, 1)
            display_frame = frame.copy()
            
            features = extract_face_features(frame, face_mesh)
            
            if features:
                try:
                    proba = clf.predict_proba([features])[0]
                    best_class_idx = np.argmax(proba)
                    best_prob = proba[best_class_idx]
                    best_name = le.inverse_transform([best_class_idx])[0]
                    
                    if best_prob > 0.7 and users_df[users_df['username'] == best_name]['is_admin'].iloc[0]:
                        results = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                        if results.multi_face_landmarks:
                            for face_landmarks in results.multi_face_landmarks:
                                mp_drawing.draw_landmarks(
                                    image=display_frame,
                                    landmark_list=face_landmarks,
                                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                                    landmark_drawing_spec=None,
                                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
                        
                        label = f"Admin: {best_name} ({best_prob*100:.1f}%)"
                        cv2.putText(display_frame, label, (10, 30), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                        cv2.putText(display_frame, "Press 'n' to continue", 
                                   (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                        
                        key = cv2.waitKey(1)
                        if key == ord('n'):
                            username = best_name
                            print(f"Recognized Admin: {username} with {best_prob*100:.1f}% confidence")
                            face_detected = True
                            break
                except Exception as e:
                    print(f"Error during face recognition: {e}")
                    logging.error(f"Face recognition error: {e}")
                    cv2.putText(display_frame, "Recognition error", 
                               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                cv2.putText(display_frame, "Show face to camera", 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            cv2.imshow("Admin Face Authentication", display_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    if not face_detected:
        cap.release()
        cv2.destroyAllWindows()
        print("Admin face recognition failed or timeout")
        logging.error("Admin face recognition failed or timeout")
        return None
    
    if username not in users_df['username'].values:
        print("User not found in database!")
        logging.error(f"User not found in database: {username}")
        cap.release()
        cv2.destroyAllWindows()
        return None
    
    user_data = users_df[users_df['username'] == username].iloc[0]
    required_sequence = eval(user_data['gesture_sequence'])
    current_sequence = []
    current_gesture = None
    gesture_start_time = None
    gesture_hold_time = 0.5
    sequence_start_time = time.time()
    
    with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=1,
        min_detection_confidence=0.7,
        min_tracking_confidence=0.5) as hands:
        
        while len(current_sequence) < len(required_sequence) and time.time() - sequence_start_time < 90:
            ret, frame = cap.read()
            if not ret:
                continue
            
            frame = cv2.flip(frame, 1)
            display_frame = frame.copy()
            
            landmarks = extract_hand_landmarks(frame, hands)
            
            if landmarks:
                hand_results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                if hand_results.multi_hand_landmarks:
                    hand_landmarks = hand_results.multi_hand_landmarks[0]
                    mp_drawing.draw_landmarks(
                        display_frame, 
                        hand_landmarks, 
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style())
                
                detected_gesture = recognize_gesture(landmarks)
                
                if detected_gesture:
                    cv2.putText(display_frame, f"Gesture Recognized: {detected_gesture}", 
                               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    
                    if len(current_sequence) < len(required_sequence):
                        expected_gesture = required_sequence[len(current_sequence)]
                        
                        if detected_gesture == expected_gesture:
                            if current_gesture != detected_gesture:
                                current_gesture = detected_gesture
                                gesture_start_time = time.time()
                            
                            hold_time = time.time() - gesture_start_time
                            
                            if hold_time >= gesture_hold_time:
                                current_sequence.append(current_gesture)
                                current_gesture = None
                                print(f"✓ Gesture {len(current_sequence)}: {current_sequence[-1]}")
                                time.sleep(0.5)
                        else:
                            current_gesture = None
                            gesture_start_time = None
            
            cv2.imshow("Admin Gesture Authentication", display_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    cap.release()
    cv2.destroyAllWindows()
    
    if current_sequence == required_sequence:
        print("\n✔ Admin authentication successful!")
        logging.info(f"Admin authentication successful for {username}")
        return username
    else:
        print("\n✖ Admin authentication failed - Wrong sequence or timeout!")
        logging.error(f"Admin authentication failed for {username}")
        return None

In [16]:
# Function to authenticate users using face recognition and gesture sequence
def authenticate_user():
    """Authenticate user with face recognition and gesture sequence"""
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open camera.")
        logging.error("Camera failed to open during user authentication")
        return False
    
    try:
        users_df = pd.read_excel('user_data/users.xlsx')
    except Exception as e:
        print(f"Error reading user database: {e}")
        logging.error(f"Error reading user database: {e}")
        return False
    
    if users_df.empty:
        print("No users registered yet!")
        logging.warning("No users registered for authentication")
        return False
    
    le, clf = load_face_recognition_model()
    
    if le is None or clf is None:
        print("Face recognition model not trained yet!")
        logging.warning("Face recognition model not trained")
        return False
    
    username = None
    
    print("Face Authentication - Show your face clearly")
    face_detected = False
    start_time = time.time()
    
    with mp_face_mesh.FaceMesh(
        static_image_mode=False,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5) as face_mesh:
        
        while time.time() - start_time < 40:
            ret, frame = cap.read()
            if not ret:
                continue
            
            frame = cv2.flip(frame, 1)
            display_frame = frame.copy()
            
            features = extract_face_features(frame, face_mesh)
            
            if features:
                try:
                    proba = clf.predict_proba([features])[0]
                    best_class_idx = np.argmax(proba)
                    best_prob = proba[best_class_idx]
                    best_name = le.inverse_transform([best_class_idx])[0]
                    
                    if best_prob > 0.7:
                        results = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                        if results.multi_face_landmarks:
                            for face_landmarks in results.multi_face_landmarks:
                                mp_drawing.draw_landmarks(
                                    image=display_frame,
                                    landmark_list=face_landmarks,
                                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                                    landmark_drawing_spec=None,
                                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
                        
                        label = f"{best_name} ({best_prob*100:.1f}%)"
                        cv2.putText(display_frame, label, (10, 30), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                        cv2.putText(display_frame, "Press 'n' to continue", 
                                   (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                        
                        key = cv2.waitKey(1)
                        if key == ord('n'):
                            username = best_name
                            print(f"Recognized: {username} with {best_prob*100:.1f}% confidence")
                            face_detected = True
                            break
                except Exception as e:
                    print(f"Error during face recognition: {e}")
                    logging.error(f"Face recognition error: {e}")
                    cv2.putText(display_frame, "Recognition error", 
                               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                cv2.putText(display_frame, "Show face to camera", 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            cv2.imshow("Face Authentication", display_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    if not face_detected:
        cap.release()
        cv2.destroyAllWindows()
        print("Face recognition failed or timeout")
        logging.error("Face recognition failed or timeout")
        return False
    
    if username not in users_df['username'].values:
        print("User not found in database!")
        logging.error(f"User not found in database: {username}")
        cap.release()
        cv2.destroyAllWindows()
        return False
    
    user_data = users_df[users_df['username'] == username].iloc[0]
    required_sequence = eval(user_data['gesture_sequence'])
    current_sequence = []
    current_gesture = None
    gesture_start_time = None
    gesture_hold_time = 0.5
    sequence_start_time = time.time()
    
    with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=1,
        min_detection_confidence=0.7,
        min_tracking_confidence=0.5) as hands:
        
        while len(current_sequence) < len(required_sequence) and time.time() - sequence_start_time < 30:
            ret, frame = cap.read()
            if not ret:
                continue
            
            frame = cv2.flip(frame, 1)
            display_frame = frame.copy()
            
            landmarks = extract_hand_landmarks(frame, hands)
            
            if landmarks:
                hand_results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                if hand_results.multi_hand_landmarks:
                    hand_landmarks = hand_results.multi_hand_landmarks[0]
                    mp_drawing.draw_landmarks(
                        display_frame, 
                        hand_landmarks, 
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style())
                
                detected_gesture = recognize_gesture(landmarks)
                
                if detected_gesture:
                    cv2.putText(display_frame, f"Gesture Recognized: {detected_gesture}", 
                               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    
                    if len(current_sequence) < len(required_sequence):
                        expected_gesture = required_sequence[len(current_sequence)]
                        
                        if detected_gesture == expected_gesture:
                            if current_gesture != detected_gesture:
                                current_gesture = detected_gesture
                                gesture_start_time = time.time()
                            
                            hold_time = time.time() - gesture_start_time
                            
                            if hold_time >= gesture_hold_time:
                                current_sequence.append(current_gesture)
                                current_gesture = None
                                print(f"✓ Gesture {len(current_sequence)}: {current_sequence[-1]}")
                                time.sleep(0.5)
                        else:
                            current_gesture = None
                            gesture_start_time = None
            
            cv2.imshow("Gesture Authentication", display_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    cap.release()
    cv2.destroyAllWindows()
    
    if current_sequence == required_sequence:
        print("\n✔ Authentication successful!")
        # Load welcome.html template
        try:
            with open('welcome.html', 'r') as f:
                html_content = f.read()
            # Replace {{username}} with actual username
            html_content = html_content.replace('{{username}}', username)
            # Write to a temporary file
            temp_html_path = 'temp_welcome.html'
            with open(temp_html_path, 'w') as f:
                f.write(html_content)
            # Open the temporary file in browser
            webbrowser.open('file://' + os.path.realpath(temp_html_path))
            logging.info(f"User authentication successful for {username}")
            return True
        except Exception as e:
            print(f"Error displaying welcome page: {e}")
            logging.error(f"Error displaying welcome page for {username}: {e}")
            return False
    else:
        print("\n✖ Authentication failed - Wrong sequence or timeout!")
        logging.error(f"User authentication failed for {username}")
        return False

In [17]:
# Function to view gesture sequence after face and OTP authentication
def view_gesture_sequence():
    """Allow a user to view their gesture sequence after face and OTP authentication"""
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open camera.")
        logging.error("Camera failed to open during gesture sequence view")
        return False
    
    try:
        users_df = pd.read_excel('user_data/users.xlsx')
    except Exception as e:
        print(f"Error reading user database: {e}")
        logging.error(f"Error reading user database: {e}")
        return False
    
    if users_df.empty:
        print("No users registered yet!")
        logging.warning("No users registered for gesture sequence view")
        return False
    
    le, clf = load_face_recognition_model()
    
    if le is None or clf is None:
        print("Face recognition model not trained yet!")
        logging.warning("Face recognition model not trained")
        return False
    
    username = None
    
    print("Face Authentication - Show your face clearly to view your gesture sequence")
    face_detected = False
    start_time = time.time()
    
    with mp_face_mesh.FaceMesh(
        static_image_mode=False,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5) as face_mesh:
        
        while time.time() - start_time < 60:
            ret, frame = cap.read()
            if not ret:
                continue
            
            frame = cv2.flip(frame, 1)
            display_frame = frame.copy()
            
            features = extract_face_features(frame, face_mesh)
            
            if features:
                try:
                    proba = clf.predict_proba([features])[0]
                    best_class_idx = np.argmax(proba)
                    best_prob = proba[best_class_idx]
                    best_name = le.inverse_transform([best_class_idx])[0]
                    
                    if best_prob > 0.7:
                        results = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                        if results.multi_face_landmarks:
                            for face_landmarks in results.multi_face_landmarks:
                                mp_drawing.draw_landmarks(
                                    image=display_frame,
                                    landmark_list=face_landmarks,
                                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                                    landmark_drawing_spec=None,
                                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
                        
                        label = f"{best_name} ({best_prob*100:.1f}%)"
                        cv2.putText(display_frame, label, (10, 30), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                        cv2.putText(display_frame, "Press 'n' to continue", 
                                   (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                        
                        key = cv2.waitKey(1)
                        if key == ord('n'):
                            username = best_name
                            print(f"Recognized: {username} with {best_prob*100:.1f}% confidence")
                            face_detected = True
                            break
                except Exception as e:
                    print(f"Error during face recognition: {e}")
                    logging.error(f"Face recognition error: {e}")
                    cv2.putText(display_frame, "Recognition error", 
                               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                cv2.putText(display_frame, "Show face to camera", 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            cv2.imshow("Face Authentication", display_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    cap.release()
    cv2.destroyAllWindows()
    
    if not face_detected:
        print("Face recognition failed or timeout")
        logging.error("Face recognition failed or timeout")
        return False
    
    if username not in users_df['username'].values:
        print("User not found in database!")
        logging.error(f"User not found in database: {username}")
        return False
    
    # OTP Verification
    user_data = users_df[users_df['username'] == username].iloc[0]
    user_email = user_data['email']
    
    if not user_email:
        print(f"No email registered for {username}. Cannot send OTP.")
        logging.error(f"No email registered for {username}")
        return False
    
    otp = generate_otp()
    otp_sent = send_otp_email(user_email, otp)
    
    if not otp_sent:
        print("Failed to send OTP. Aborting.")
        logging.error(f"Failed to send OTP to {user_email}")
        return False
    
    print(f"\nAn OTP has been sent to {user_email}. Please check your email (including spam/junk folder).")
    otp_timeout = time.time() + 300  # 5 minutes timeout
    
    attempts = 3
    while attempts > 0 and time.time() < otp_timeout:
        user_otp = input(f"Enter the OTP ({attempts} attempts remaining): ").strip()
        if user_otp == otp:
            print("OTP verified successfully!")
            logging.info(f"OTP verified for {username}")
            break
        else:
            print("Invalid OTP. Please try again.")
            logging.warning(f"Invalid OTP attempt for {username}")
            attempts -= 1
    
    if attempts == 0 or time.time() >= otp_timeout:
        print("OTP verification failed or timed out.")
        logging.error(f"OTP verification failed or timed out for {username}")
        return False
    
    # Display gesture sequence
    gesture_sequence = eval(user_data['gesture_sequence'])
    
    print(f"\nYour gesture sequence, {username}:")
    print(" -> ".join(gesture_sequence))
    print("\nGesture Instructions:")
    for gesture in gesture_sequence:
        print(f"- {gesture}: {GESTURE_INSTRUCTIONS[gesture]}")
    
    logging.info(f"Gesture sequence viewed by {username}")
    return True

In [18]:
# Function to delete a registered user only by admin's authentication and confirmation
def delete_user():
    """Delete a registered user (admin only)"""
    admin_username = authenticate_admin()
    if not admin_username:
        print("Admin authentication failed. Cannot delete user.")
        logging.error("Admin authentication failed for user deletion")
        return False
    
    try:
        users_df = pd.read_excel('user_data/users.xlsx')
    except Exception as e:
        print(f"Error reading user database: {e}")
        logging.error(f"Error reading user database: {e}")
        return False
    
    if users_df.empty:
        print("No users registered!")
        logging.warning("No users registered for deletion")
        return False
    
    print("\nRegistered users:")
    print(users_df['username'].to_string(index=False))
    
    username = input("\nEnter username to delete: ").strip()
    if username not in users_df['username'].values:
        print("User not found!")
        logging.warning(f"Attempted to delete non-existent user: {username}")
        return False
    
    if username == admin_username:
        print("Admins cannot delete themselves!")
        logging.warning(f"Admin {admin_username} attempted to delete self")
        return False
    
    user_data = users_df[users_df['username'] == username].iloc[0]
    try:
        if os.path.exists(user_data['face_encoding_file']):
            os.remove(user_data['face_encoding_file'])
    except Exception as e:
        print(f"Error deleting face encoding file: {e}")
        logging.error(f"Error deleting face encoding file for {username}: {e}")
        return False
    
    users_df = users_df[users_df['username'] != username]
    
    try:
        users_df.to_excel('user_data/users.xlsx', index=False)
        print(f"User {username} deleted successfully by admin {admin_username}!")
        logging.info(f"User {username} deleted by admin {admin_username}")
        
        if len(users_df) >= 2:
            update_face_recognition_model()
        return True
    except Exception as e:
        print(f"Error saving updated user database: {e}")
        logging.error(f"Error saving updated user database: {e}")
        return False

In [19]:
# Function to clear all user data only by admin's authentication and confirmation
def clear_database():
    """Clear all user data (admin only)"""
    admin_username = authenticate_admin()
    if not admin_username:
        print("Admin authentication failed. Cannot clear database.")
        logging.error("Admin authentication failed for database clear")
        return False
    
    confirmation = input("Are you sure you want to clear ALL user data? (y/n): ").lower()
    if confirmation != 'y':
        logging.info("Database clear cancelled")
        return False
    
    try:
        if os.path.exists('user_data/users.xlsx'):
            df = pd.DataFrame(columns=['username', 'email', 'gesture_sequence', 'face_encoding_file', 'is_admin'])
            df.to_excel('user_data/users.xlsx', index=False)
        
        for file in os.listdir('user_data/face_encodings'):
            os.remove(f"user_data/face_encodings/{file}")
        
        if os.path.exists('models/face_recognition_model.pkl'):
            os.remove('models/face_recognition_model.pkl')
        
        print(f"All user data cleared successfully by admin {admin_username}!")
        logging.info(f"All user data cleared by admin {admin_username}")
        return True
    except Exception as e:
        print(f"Error clearing database: {e}")
        logging.error(f"Error clearing database: {e}")
        return False

In [20]:
# Function to restart the application using a simple HTTP server
class RestartHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        if self.path == '/restart':
            self.send_response(200)
            self.end_headers()
            self.wfile.write(b"Restarting...")
            subprocess.Popen([sys.executable] + sys.argv)
            sys.exit(0)
        else:
            self.send_response(404)
            self.end_headers()

In [21]:
def start_server():
    server = HTTPServer(('localhost', 8000), RestartHandler)
    server.serve_forever()

In [22]:
# Main function to run the multi-factor authentication system
def main():
    """Main menu"""
    server_thread = threading.Thread(target=start_server, daemon=True)
    server_thread.start()
    
    while True:
        print("\n===== Multi-Factor Authentication System =====")
        print("1. Register New User")
        print("2. Authenticate User")
        print("3. View Gesture Sequence")
        print("4. Delete User (Admin Only)")
        print("5. Clear Database (Admin Only)")
        print("6. Exit")
        
        choice = input("Enter your choice: ").strip()
        
        if choice == '1':
            register_user()
        elif choice == '2':
            authenticate_user()
        elif choice == '3':
            view_gesture_sequence()
        elif choice == '4':
            delete_user()
        elif choice == '5':
            clear_database()
        elif choice == '6':
            print("Exiting...")
            logging.info("Application exited")
            break
        else:
            print("Invalid choice! Please try again.")
            logging.warning(f"Invalid menu choice: {choice}")

In [23]:
if __name__ == "__main__":
    main()


===== Multi-Factor Authentication System =====
1. Register New User
2. Authenticate User
3. View Gesture Sequence
4. Delete User (Admin Only)
5. Clear Database (Admin Only)
6. Exit
Admin Face Authentication - Show your face clearly
Admin face recognition failed or timeout
Admin authentication failed. Cannot delete user.

===== Multi-Factor Authentication System =====
1. Register New User
2. Authenticate User
3. View Gesture Sequence
4. Delete User (Admin Only)
5. Clear Database (Admin Only)
6. Exit
Admin Face Authentication - Show your face clearly
Recognized Admin: Raghavendra with 74.7% confidence
✓ Gesture 1: open_hand
✓ Gesture 2: rock_on

✖ Admin authentication failed - Wrong sequence or timeout!
Admin authentication failed. Cannot clear database.

===== Multi-Factor Authentication System =====
1. Register New User
2. Authenticate User
3. View Gesture Sequence
4. Delete User (Admin Only)
5. Clear Database (Admin Only)
6. Exit
Admin Face Authentication - Show your face clearly
Adm